# All the imports in one place

In [656]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import PolynomialFeatures



path = "data/properties.csv"
df = pd.read_csv(path)


In [657]:
nan_counts = df.isna().sum()
nan_counts = nan_counts.sort_values(ascending=False)
nan_counts[nan_counts > 0]

cadastral_income                  44967
surface_land_sqm                  36256
construction_year                 33391
primary_energy_consumption_sqm    26567
nbr_frontages                     26346
latitude                          14098
longitude                         14098
terrace_sqm                       13140
total_area_sqm                     7615
garden_sqm                         2939
dtype: int64

In [658]:
df.columns.tolist()

['id',
 'price',
 'property_type',
 'subproperty_type',
 'region',
 'province',
 'locality',
 'zip_code',
 'latitude',
 'longitude',
 'construction_year',
 'total_area_sqm',
 'surface_land_sqm',
 'nbr_frontages',
 'nbr_bedrooms',
 'equipped_kitchen',
 'fl_furnished',
 'fl_open_fire',
 'fl_terrace',
 'terrace_sqm',
 'fl_garden',
 'garden_sqm',
 'fl_swimming_pool',
 'fl_floodzone',
 'state_building',
 'primary_energy_consumption_sqm',
 'epc',
 'heating_type',
 'fl_double_glazing',
 'cadastral_income']

# Class for Preprocessing

In [659]:
class Preprocessing:

    @staticmethod
    def fill_nan_with_median(df, column_name):
        """
        Fill missing values in the specified column(s) with the median value.

        Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        column_name (str or list): The name(s) of the column(s) to fill.

        Returns:
        None
        """
        if isinstance(column_name, list):
            for column in column_name:
                median_of_column = df[column].median()
                df.fillna({column: median_of_column}, inplace=True)
        else:
            median_of_column = df[column_name].median()
            df.fillna({column_name: median_of_column}, inplace=True)

        return None


    @staticmethod
    def fill_nan_with_0(df, column_name):
        """
        Fill NaN values in the specified column(s) with 0.

        Parameters:
        - df (pandas.DataFrame): The DataFrame containing the data.
        - column_name (str or list): The name(s) of the column(s) to fill NaN values in.

        Returns:
        None
        """
        if isinstance(column_name, list):
            for column in column_name:
                df.fillna({column: 0}, inplace=True)
        else:
            df.fillna({column_name: 0}, inplace=True)

        return None
    
    @staticmethod
    def iqr(dataframe, column_names):
        """
        Filter the given dataframe based on the interquartile range (IQR) of the specified column(s).

        Parameters:
        - dataframe: pandas DataFrame
            The input dataframe to filter.
        - column_names: str or list of str
            The name(s) of the column(s) to calculate the IQR and perform the filtering.

        Returns:
        - filtered_df: pandas DataFrame
            The filtered dataframe based on the IQR of the specified column(s).
        """
        filtered_df = dataframe.copy()

        if isinstance(column_names, list):
            for column_name in column_names:
                Q1 = dataframe[column_name].quantile(0.25)
                Q3 = dataframe[column_name].quantile(0.75)
                IQR = Q3 - Q1

                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR

                filtered_df = filtered_df[(filtered_df[column_name] >= lower_bound) & (filtered_df[column_name] <= upper_bound)]
        else:
            Q1 = dataframe[column_names].quantile(0.25)
            Q3 = dataframe[column_names].quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            filtered_df = filtered_df[(filtered_df[column_names] >= lower_bound) & (filtered_df[column_names] <= upper_bound)]

        return filtered_df
    

    def iqr(dataframe, column_names, k=1.5):
        """
        Filter the given dataframe based on the interquartile range (IQR) of the specified column(s).

        Parameters:
        - dataframe: pandas DataFrame
            The input dataframe to filter.
        - column_names: str or list of str
            The name(s) of the column(s) to calculate the IQR and perform the filtering.
        - k: float
            The factor to multiply the IQR by when calculating the lower and upper bounds.

        Returns:
        - filtered_df: pandas DataFrame
            The filtered dataframe based on the IQR of the specified column(s).
        """
        filtered_df = dataframe.copy()

        if isinstance(column_names, list):
            for column_name in column_names:
                Q1 = dataframe[column_name].quantile(0.25)
                Q3 = dataframe[column_name].quantile(0.75)
                IQR = Q3 - Q1

                lower_bound = Q1 - k * IQR
                upper_bound = Q3 + k * IQR

                filtered_df = filtered_df[(filtered_df[column_name] >= lower_bound) & (filtered_df[column_name] <= upper_bound)]
        else:
            Q1 = dataframe[column_names].quantile(0.25)
            Q3 = dataframe[column_names].quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            filtered_df = filtered_df[(filtered_df[column_names] >= lower_bound) & (filtered_df[column_names] <= upper_bound)]

        return filtered_df

prep = Preprocessing

# Model: Appartaments and Houses

### Preprocessing

In [660]:
# Logical fixing

In [661]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

df_train.dropna(subset=['total_area_sqm'], inplace=True)
df_train.dropna(subset=['construction_year'], inplace=True)
df_train.dropna(subset=['nbr_frontages'], inplace=True)
df_train.dropna(subset=['garden_sqm'], inplace=True)
df_train.dropna(subset=['primary_energy_consumption_sqm'], inplace=True)

# Create train features here

df_train['total_energy_consumption'] = df_train.primary_energy_consumption_sqm / df_train.total_area_sqm

selected_features_train = df_train[[
    'price', 'nbr_bedrooms', 'total_area_sqm', 'region', 'subproperty_type', 'garden_sqm', 'nbr_frontages', 'primary_energy_consumption_sqm'
]]

# Create test features here

df_train['total_energy_consumption'] = df_train.primary_energy_consumption_sqm / df_train.total_area_sqm

X_test = df_test.drop(['price'], axis=1)
y_test = df_test['price']

selected_features_train = prep.iqr(selected_features_train,
                                   ['price', 'nbr_bedrooms', ],
                                    k=1.5)

# Drop price_per_sqm from the features as it's derived from the target
X_train = selected_features_train.drop(['price'], axis=1)
y_train = selected_features_train['price']

numeric_sqm_features = []

categorical_columns = selected_features_train.select_dtypes(include=['object']).columns.tolist()

numeric_features = [col for col in X_train.columns if col not in categorical_columns and col not in numeric_sqm_features]


In [662]:
nan_counts = df_train.isna().sum()
nan_counts = nan_counts.sort_values(ascending=False)
nan_counts[nan_counts > 0]

surface_land_sqm    6185
cadastral_income    5577
terrace_sqm         3922
latitude            2991
longitude           2991
dtype: int64

In [663]:
df_train.shape[0]

15606

# Model Pipeline

In [664]:
numeric_sqm_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_features),
        ('numeric_sqm', numeric_sqm_transformer, numeric_sqm_features),
        ('categorical', categorical_transformer, categorical_columns)])

model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('model', LinearRegression())])

model_pipeline.fit(X_train, y_train)

model_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['nbr_bedrooms',
                                                   'total_area_sqm',
                                                   'garden_sqm',
                                                   'nbr_frontages',
                                                   'primary_energy_consumption_sqm']),
                                                 ('numeric_sqm',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  []),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['region',
                                                   'subproperty_type'])])),
                ('model', LinearRegression())])

In [665]:
# Predicting on the test set
y_pred = model_pipeline.predict(X_test)

# Calculating MSE
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculating RMSE
rmse = np.sqrt(mse)
print("Root Mean Squared Error: €" + str(round(rmse)))

print(f'Score: {round(model_pipeline.score(X_test, y_test) * 100, 2)}%')

Mean Squared Error: 140052989824.71344
Root Mean Squared Error: €374237
Score: 19.5%
